# Introduction & Technical Set Up

The purpose of this notebook is to identify the optimal dates in which to take various kinds of parental leave in order to meet our family's goals of a) maximizing the amount of leave I can take continuously and b) front-loading our earnings during the leave period. 

I will treat this as a two-step problem: In the first step, I will identify the maximum amount of leave that I will take I can take continuously without considering when money is earned. In the second step, I will identify a leave schedule that both allows me to take approximately that amount of leave and front-loads earnings. 

In [ ]:
# Set up packages
import pulp
import pandas as pd
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Load from configuration file
load_dotenv()

In [ ]:
# Set up the problem
prob = pulp.LpProblem("Parental_Leave_Optimization", pulp.LpMaximize)

# PART I: Identifying the Maximum Amount of Continous Leave

In this step, I will identify the maximum amount of leave that I will take I can take continuously based on company policies. 

## Date Definitions and Classifications 

In this section, I will define the scope of dates includeded in the analysis and define various types of leave. I will also define company holidays.

In [ ]:
# Define dates
start_date = datetime(2025, 1, 1)
end_date = datetime(2026, 4, 1) 
due_date = datetime.strptime(os.getenv("due_date"), "%Y-%m-%d")  # Get due date from configuration file
dates = pd.date_range(start_date, end_date)

# Group dates 
weeks = dates.to_period('W-SUN').unique()
months = dates.to_period('M').unique()

# Create dictionaries to hold date classifications

# These classifications describe how my time will be classified
is_std = {} #Short term disability
is_paid_fcl = {} #Paid Family Care Leave
is_unpaid_fcl_leave = {} #Unpaid Family Care Leave
is_vacation = {} # Vacation PTO
is_optional_holiday = {} 
is_non_absence = {} # Day on which I am working
is_not_working = {} #Generic category for all days with any kind of leave, company closure, weekend, vacation, etc

# These classifications describe how the company classifies a given day
is_company_holiday = {}
is_weekend = {}
is_company_workday = {} # Generic classification for all days that are not holidays or weekends

# Hard-code company holiday definitions 
# Note: The 2026 holidays have not been added to the list yet
company_holidays = [
    #2025 Company Holidays
    datetime(2025, 1, 1),   # New Year's Day
    datetime(2025, 1, 20),  # MLK Day
    datetime(2025, 2, 17),  # Presidents Day
    datetime(2025, 5, 26),  # Memorial Day
    datetime(2025, 6, 19),  # Juneteenth
    datetime(2025, 7, 4),   # Independence Day
    datetime(2025, 9, 1),   # Labor Day
    datetime(2025, 11, 27), # Thanksgiving
    datetime(2025, 11, 28), # Day after Thanksgiving
    datetime(2025, 12, 25), # Christmas
    datetime(2025, 12, 26), # Day after Christmas
    datetime(2025, 12, 29), # Global Shutdown
    datetime(2025, 12, 30), # Global Shutdown
    datetime(2025, 12, 31),  # Global Shutdown
    # Placeholder for 2026 Company Holidays
    # The company has not yet released its 2026 holiday schedule
    # These are my assumptions based on the 2025 schedule
    datetime(2026, 1, 1),   # New Year's Day
    datetime(2026, 1, 19),  # MLK Day
    datetime(2026, 2, 16),  # Presidents Day
    datetime(2026, 5, 25),  # Memorial Day
    datetime(2026, 6, 19)  # Juneteenth
]

# Create variables for each date
for d in dates: 

    # Hard-code weekend definitions
    if d.weekday() == 5 or d.weekday() == 6:  # Saturday or Sunday
        is_weekend[d] = 1
    else:
        is_weekend[d] = 0


    if d in company_holidays and not is_weekend[d]:
        is_company_holiday[d] = 1
    else:
        is_company_holiday[d] = 0

    # Hard-code the company workdays based on the company holidays and weekends
    if not(is_company_holiday[d] or is_weekend[d]):
        is_company_workday[d] = 1
    else:
        is_company_workday[d] = 0

    # Hard-code the short-term disability dates

    if d >= datetime.strptime(os.getenv("std_start_date"), "%Y-%m-%d") and d <= datetime.strptime(os.getenv("std_end_date"), "%Y-%m-%d") and is_company_workday[d]:
            is_std[d] = 1
    else:
        is_std[d] = 0

    # Define the remaining types of days as pulp variables 
    is_paid_fcl[d] = pulp.LpVariable(f"is_paid_fcl_{d.strftime('%Y%m%d')}", cat='Binary')
    is_unpaid_fcl_leave[d] = pulp.LpVariable(f"is_unpaid_fcl_leave_{d.strftime('%Y%m%d')}", cat='Binary')
    is_vacation[d] = pulp.LpVariable(f"is_vacation_{d.strftime('%Y%m%d')}", cat='Binary')
    is_optional_holiday[d] = pulp.LpVariable(f"is_optional_holiday_{d.strftime('%Y%m%d')}", cat='Binary')
    is_non_absence[d] = pulp.LpVariable(f"is_non_absence_{d.strftime('%Y%m%d')}", cat='Binary')
    is_not_working[d] = pulp.LpVariable(f"is_not_working_{d.strftime('%Y%m%d')}", cat='Binary')

# Each date can only have one type
for d in dates:
    prob += (is_company_holiday[d] + is_weekend[d] + is_std[d] +
             is_paid_fcl[d] + is_unpaid_fcl_leave[d] + is_vacation[d] + is_optional_holiday[d] + is_non_absence[d] == 1), f"Date_Category_{d.strftime('%Y%m%d')}"

# Define the general non-working category
    prob += (
        is_not_working[d] == (
            is_company_holiday[d] + is_weekend[d] + is_std[d] +
            is_paid_fcl[d] + is_unpaid_fcl_leave[d] +
            is_vacation[d] + is_optional_holiday[d]
        )
    ), f"is_not_working_definition{d.strftime('%Y%m%d')}"


## Define the Objective Function

The overall objective is to maximize the amount of continuous parental leave that I can take after the baby is born. I define this objective in two sections. First, for each date in the analysis, I calculate the amount of leave that I have taken continuously on that date. Second, I identify the maximum amount of continuous leave taken across the entire analysis. This maximum value will be used in the objective function.    

In [ ]:

# Add variables to track the number of continuous leave days ending on each date
continuous_nonworking_days = {d: pulp.LpVariable(f"continuous_nonworking_days_{d.strftime('%Y%m%d')}", lowBound=0) for d in dates}
M = len(dates) 

# Add constraints to calculate continuous leave days
for i, d in enumerate(dates):
    if i == 0:
        # For the first day, continuous leave days is equal to is_not_working
        prob += continuous_nonworking_days[d] == is_not_working[d], f"continuous_Nonworking_Start_{d.strftime('%Y%m%d')}"
    else:
        # For other days, we use a Big M approach to calculate continuous leave days
        prev_day = dates[i - 1]
        prob += continuous_nonworking_days[d] >= continuous_nonworking_days[prev_day] + is_not_working[d] - M * (1 - is_not_working[d]), f"continuous_Nonworking_Constraint_Lower_{d.strftime('%Y%m%d')}"
        prob += continuous_nonworking_days[d] <= continuous_nonworking_days[prev_day] + is_not_working[d], f"continuous_Nonworking_Constraint_Increment_{d.strftime('%Y%m%d')}"
        prob += continuous_nonworking_days[d] <= is_not_working[d] * M, f"continuous_Nonworking_Zero_{d.strftime('%Y%m%d')}"


In [ ]:
# Add variables to track the maximum amount of continuous leave days
# This is the variable we want to maximize

# Ensure max_continuous_nonworking_days is at least as large as any continuous_nonworking_days value
max_continuous_nonworking_days = pulp.LpVariable("max_continuous_nonworking_days", lowBound=0)
M = len(dates) 

# Define binary helper variables to indicate which variable is the maximum
z = {d: pulp.LpVariable(f"z_{d.strftime('%Y%m%d')}", cat="Binary") for d in dates}

# Ensure exactly one z[d] is active
prob += pulp.lpSum(z[d] for d in dates) == 1, "One_Max_Indicator"

for d in dates:
    prob += max_continuous_nonworking_days >= continuous_nonworking_days[d], f"Max_continuous_Nonworking_{d.strftime('%Y%m%d')}"
    prob += max_continuous_nonworking_days <= continuous_nonworking_days[d] + M * (1 - z[d]), f"Max_continuous_UpperBound_{d.strftime('%Y%m%d')}"
    

In [ ]:
#Objective: Maximize the continuous amount of leave
prob += max_continuous_nonworking_days, "Maximize_continuous_Leave"

# Simplified Objective: Maximize amount of leave
# The simplified objective can be useful for debugging
#prob += pulp.lpSum([is_not_working[d] for d in dates]), "Maximize Leave"

## Define Company Policies

This section defines constraints which correspond to company policies on each kind of leave: optional holidays, paid family care leave, unpaid family care leave, and vacation days. 

In [ ]:
# There are rules that govern optional holidays:

# Rule 1: Only three optional holidays can be taken in any given year
prob += (pulp.lpSum([is_optional_holiday[d] for d in dates if d.year == 2025]) <= 3), "Max_Optional_Holidays_2025"
prob += (pulp.lpSum([is_optional_holiday[d] for d in dates if d.year == 2026]) <= 3), "Max_Optional_Holidays_2026"

# Rule 2: Hard code optional holidays prior to the due date based on history
# I did not take any optional holidays in 2025 and will not take any until after the due date
for d in dates:
    if d <= due_date:
        prob += (is_optional_holiday[d] == 0), f"Optional_Holiday_Historical_{d.strftime('%Y%m%d')}"

In [ ]:
# There are rules that govern paid family care leave

# Define binary variables for each week to indicate if the week is a paid family care leave week
is_paid_fcl_week = {w: pulp.LpVariable(f"is_paid_fcl_week_{w.start_time.strftime('%Y%m%d')}", cat='Binary') for w in weeks}

# Rule 1: If day is taken as paid family care leave, all such days in that week are counted
for w in weeks:
	week_dates = dates[(dates >= w.start_time) & (dates <= w.end_time)]
	for d in week_dates:
		prob += is_paid_fcl[d] <= is_paid_fcl_week[w], f"Paid_FCL_Week_{d.strftime('%Y%m%d')}"

# Rule 2: Paid family care leave must be taken in two-week increments
for w in weeks: 
	prob += is_paid_fcl_week[w] <= is_paid_fcl_week.get(w - 1, 0) + is_paid_fcl_week.get(w + 1, 0), f"Two_Week_Increment_Paid_FCL_{w.start_time.strftime('%Y%m%d')}"

# Rule 3: There are up to twelve weeks of paid family care leave
prob += pulp.lpSum(is_paid_fcl_week[w] for w in weeks) <= 12, "Max_Twelve_Weeks_Paid_FCL"

# Rule 4: Paid family care cannot be taken prior to due due date 
for d in dates:
	if d < due_date:
		prob += is_paid_fcl[d] == 0, f"Paid_FCL_After_Due_Date_{d.strftime('%Y%m%d')}"

# Rule 5: Paid family care leave can only be taken within a year of the due date
for d in dates:
	if d > due_date + timedelta(days=365):
		prob += is_paid_fcl[d] == 0, f"Paid_FCL_Within_Year_{d.strftime('%Y%m%d')}"

# Rule 6: If a week is taken as paid family care leave, there should be no non-absence or other leave days in that week
for w in weeks:
	week_dates = dates[(dates >= w.start_time) & (dates <= w.end_time)]
	for d in week_dates:
		prob += is_paid_fcl_week[w] + is_non_absence[d] <= 1, f"Paid_FCL_Week_is_non_absence_{d.strftime('%Y%m%d')}"
		prob += is_paid_fcl_week[w] + is_std[d] <= 1, f"Paid_FCL_Week_is_std_{d.strftime('%Y%m%d')}"
		prob += is_paid_fcl_week[w] + is_vacation[d] <= 1, f"Paid_FCL_Week_is_vacation_{d.strftime('%Y%m%d')}"
		prob += is_paid_fcl_week[w] + is_optional_holiday[d] <= 1, f"Paid_FCL_Week_is_optional_holiday_{d.strftime('%Y%m%d')}"
		prob += is_paid_fcl_week[w] + is_unpaid_fcl_leave[d] <= 1, f"Paid_FCL_Week_is_unpaid_fcl_{d.strftime('%Y%m%d')}"

In [ ]:
# There are rules that govern unpaid family care leave

# Define binary variables for each week to indicate if the week is a unpaid family care leave week
is_unpaid_fcl_week = {w: pulp.LpVariable(f"is_unpaid_fcl_week_{w.start_time.strftime('%Y%m%d')}", cat='Binary') for w in weeks}

# Rule 1: If in a week is taken as paid family care leave, all such days in that week are counted
for w in weeks:
	week_dates = dates[(dates >= w.start_time) & (dates <= w.end_time)]
	for d in week_dates:
		prob += is_unpaid_fcl_leave[d] <= is_unpaid_fcl_week[w], f"Unpaid_FCL_Week_{d.strftime('%Y%m%d')}"
			
# Rule 2: Paid family care leave must be taken in two-week increments
for w in weeks: 
	prob += is_unpaid_fcl_week[w] <= is_unpaid_fcl_week.get(w - 1, 0) + is_unpaid_fcl_week.get(w + 1, 0), f"Two_Week_Increment_Unpaid_FCL{w.start_time.strftime('%Y%m%d')}"

# Rule 3: There are up to twelve weeks of unpaid family care leave
prob += pulp.lpSum(is_unpaid_fcl_week[w] for w in weeks) <= 16, "Max_Sixteen_Weeks_Unpaid_FCL"

# Rule 4: Paid family care leave cannot be taken prior to due due date 
for d in dates:
	if d < due_date:
		prob += is_unpaid_fcl_leave[d] == 0, f"Unpaid_FCL_After_Due_Date_{d.strftime('%Y%m%d')}"

# Rule 5: Paid family care leave leave can only be taken within a year of the due date
for d in dates:
	if d > due_date + timedelta(days=365):
		prob += is_unpaid_fcl_leave[d] == 0, f"Unpaid_FCL_Within_Year_{d.strftime('%Y%m%d')}"

# Rule 6: If a week is taken as unpaid family care leave, there should be no non-absence or other leave days in that week
for w in weeks:
	week_dates = dates[(dates >= w.start_time) & (dates <= w.end_time)]
	for d in week_dates:
		prob += is_unpaid_fcl_week[w] + is_non_absence[d] <= 1, f"Unpaid_FCL_Week_is_non_absence_{d.strftime('%Y%m%d')}"
		prob += is_unpaid_fcl_week[w] + is_std[d] <= 1, f"Unpaid_FCL_Week_is_std_{d.strftime('%Y%m%d')}"
		prob += is_unpaid_fcl_week[w] + is_vacation[d] <= 1, f"Unpaid_FCL_Week_is_vacation_{d.strftime('%Y%m%d')}"
		prob += is_unpaid_fcl_week[w] + is_optional_holiday[d] <= 1, f"Unpaid_FCL_Week_is_optional_holiday_{d.strftime('%Y%m%d')}"
		prob += is_unpaid_fcl_week[w] + is_paid_fcl[d] <= 1, f"Unpaid_FCL_Week_is_paid_fcl_{d.strftime('%Y%m%d')}"

In [ ]:

# There are rules that govern how vacation days are accrued and taken
# Note that I have implemented a simplified version of the accrual rules for 2026

# Rule 1: Vacation days prior to due date should reflect the historical vacation days taken 
vacation_days_taken = [
    datetime(2025, 2, 21) # Note that this was a half-day; modeling as a full day for simplicity
    ]

for d in dates:
    # Hard-code vacation days taken prior to due date
    if d in vacation_days_taken:
        prob += (is_vacation[d] == 1), f"Vacation_Historical_{d.strftime('%Y%m%d')}"
    # Hard code vacation days not taken prior to due date 
    elif d <= due_date:
        prob += (is_vacation[d] == 0), f"Vacation_Historical_{d.strftime('%Y%m%d')}"

#Rule 2: I rolled over 40 hours (5 days) of vacation from 2024 to 2025
vacation_days_rolled_over_2024_to_2025 = 5

# Rule 3: The formula by which vacation days are accrued is complex.
# The rule is that vacation days are accrued at the rate of 10% of the annual allotment for each full calendar monnth worked.
# Full annual allotment is earned by working 10 months out of the calendar year. An employee will be
# considered to have worked a full calendar month if (s)he works at least 50% of his/her regular work schedule.
# The definition of worked days is not intuitive since it includes some leave types.

# Note that I built these rules out for 2025 only since I do not plan on taking extended leave in 2026.
# If I do take significant leave in 2026, I will need to modify the model to reflect that.

annual_vacation_allotment_days = 15 # 15 days
vacation_days_accrued_2025_per_month = annual_vacation_allotment_days * 0.1
is_vacation_accrued_in_month = {m: pulp.LpVariable(f"is_vacation_accrued_month_{m.start_time.strftime('%Y%m%d')}", cat='Binary') for m in months}
proportion_of_month_worked = {m: pulp.LpVariable(f"proportion_of_month_worked_{m.start_time.strftime('%Y%m%d')}") for m in months}

vacation_days_accrued_2025 = pulp.LpVariable("vacation_days_accrued_2025", lowBound=0, upBound= annual_vacation_allotment_days)
is_full_annual_vacation_allotment_accrued_2025 = pulp.LpVariable("is_full_annual_vacation_allotment_accrued_2025", cat='Binary')

is_day_accrual_eligible = {d: pulp.LpVariable(f"is_day_accrual_eligible_{d.strftime('%Y%m%d')}", cat='Binary') for d in dates}
for d in dates:
    prob += is_day_accrual_eligible[d] == is_std[d] + is_paid_fcl[d] + is_vacation[d] + is_optional_holiday[d] + is_non_absence[d], f"Day_Accrual_Eligible_{d.strftime('%Y%m%d')}"

for m in months:
    # For simplicity, we will ignore vacation accrual in 2026
    # My assumption is that I will work at least 10 months and will get the full allotment. 
    if m.year == 2026:
        pass 
    else:
        month_dates = dates[(dates >= m.start_time) & (dates <= m.end_time)]

        #Calculate the proportion of the month worked
        workdays_per_month = len([d for d in month_dates if is_company_workday[d] == 1])
        prob += proportion_of_month_worked[m] == pulp.lpSum(is_day_accrual_eligible[d] for d in month_dates) / workdays_per_month   

        # Add Big-M constraints to enforce the conditions that govern whether a month is eligible for vacation accrual
        M = 1 
        # If proportion_of_month_worked[m] >= 0.5, then is_vacation_accrued_in_month[m] = 1
        prob += (
            proportion_of_month_worked[m] >= 0.5 - M * (1 - is_vacation_accrued_in_month[m]),
            f"Vacation_Accrual_Min_{m.start_time.strftime('%Y%m')}"
        )
    
        # If proportion_of_month_worked[m] < 0.5, then is_vacation_accrued_in_month[m] = 0
        prob += (
            proportion_of_month_worked[m] <= 0.5 + M * is_vacation_accrued_in_month[m],
            f"Vacation_Accrual_Max_{m.start_time.strftime('%Y%m')}"
        )

# Add constraints to define the total number of vacation days accrued
num_vacation_eligible_months_2025 = pulp.LpVariable("num_vacation_eligible_months_2025", lowBound=0)
num_vacation_eligible_months_2025 = pulp.lpSum(is_vacation_accrued_in_month[m] for m in months if m.year == 2025)
M = 13

prob += num_vacation_eligible_months_2025 <= 9.99 + M * is_full_annual_vacation_allotment_accrued_2025, "is_full_annual_vacation_allotment_accrued_2025_Upper"
prob += num_vacation_eligible_months_2025 >= 10 - M * (1 - is_full_annual_vacation_allotment_accrued_2025), "is_full_annual_vacation_allotment_accrued_2025_Lower"

# If num_vacation_eligible_months_2025 < 10, accrue monthly
prob += vacation_days_accrued_2025 >= num_vacation_eligible_months_2025 * vacation_days_accrued_2025_per_month - M * is_full_annual_vacation_allotment_accrued_2025
prob += vacation_days_accrued_2025 <= num_vacation_eligible_months_2025 * vacation_days_accrued_2025_per_month + M * is_full_annual_vacation_allotment_accrued_2025

# If num_vacation_eligible_months_2025 >= 10, get full allotment
prob += vacation_days_accrued_2025 >= annual_vacation_allotment_days - M * (1 - is_full_annual_vacation_allotment_accrued_2025)
prob += vacation_days_accrued_2025 <= annual_vacation_allotment_days + M * (1 - is_full_annual_vacation_allotment_accrued_2025)

# Rule 4: Vacation days can be rolled over from 2025 to 2026
# The actual rule is that up to 40 hours (5 days) can be rolled over
# However, I am going to handle the amount of rollover as a preference 
# My goal here is simply to define the rollover variable 
vacation_days_rolled_over_2025_to_2026 = pulp.LpVariable("vacation_days_rolled_over_2025_to_2026", lowBound=0)
prob += vacation_days_rolled_over_2025_to_2026 == vacation_days_accrued_2025 + vacation_days_rolled_over_2024_to_2025 - pulp.lpSum([is_vacation[d] for d in dates if d.year == 2025]), "Rollover_Vacation_Days_2025_to_2026"

# Rule 4: The number of vacation days taken in a given year cannot exceed the number of vacation days available in 2025
# For 2025, we will calculate the vacation days available based on accrual. For 2026, we will assume that the full allotment is available.
prob += (pulp.lpSum([is_vacation[d] for d in dates if d.year == 2025]) <= vacation_days_accrued_2025 + vacation_days_rolled_over_2024_to_2025), "Max_Vacation_Days_Taken"
prob += (pulp.lpSum([is_vacation[d] for d in dates if d.year == 2026]) <= annual_vacation_allotment_days + vacation_days_rolled_over_2025_to_2026), "Max_Vacation_Days_Taken_2026"

## Personal Preferences

In this section, I create constraints that reflect my own preferences. These can be adjusted or toggled on and off as needed. It may be worthwhile to experiment with removing or loosening some of these. 

In [ ]:
# These are my preferences for how I would like to take leave
# For simplicity, I am hard-coding these as constraints rather than adding penalties to the objective function

# Preference 1: I do not want to work between the due date and our day-care start date
for d in dates:
    if d >= due_date and d <= datetime.strptime(os.getenv("no_work_before_date"), "%Y-%m-%d"):
        prob += (is_not_working[d] == 1), f"No_work_before_this_date_{d.strftime('%Y%m%d')}"

# Preference 2: I would like to take all of my paid parental leave
prob += pulp.lpSum(is_paid_fcl_week[w] for w in weeks) == 12, "Twelve_Weeks_Paid_FCL"

# Preference 3: I want to use up a limited amount of vacation and optional holiday in 2026
# It would be nice to save this time periods outside the scope of this model (ie, fall or summer vacation)
prob += pulp.lpSum([is_vacation[d] for d in dates if d.year == 2026]) + pulp.lpSum([is_optional_holiday[d] for d in dates if d.year == 2026]) <= 1, "Save_2026_Time_Off"

# Preference 4: I would like to roll-over at least 4.5 days from 2025 to 2026
# Note that there is no benefit to rolling over more time
prob += vacation_days_rolled_over_2025_to_2026 >= 4.5, "Rollover_Vacation_Days_2025_to_2026_Min"

# Preference 5: We may be constrained in the amount of unpaid leave I can take beyond the constraints
# defined by company policy. This constraint can be set via the configuration file. 
prob += pulp.lpSum(is_unpaid_fcl_week[w] for w in weeks) <= float(os.getenv("max_weeks_unpaid_leave")), "max_weeks_unpaid_leave_preference"

# PLACEHOLDER: We may be constrained in the amount of unpaid leave we can take before a certain date. 
# We should experiment with this to understand the impact. 
# I will table this until I know more about short-term disability dates and our financial needs.

In [ ]:
prob.solve(pulp.PULP_CBC_CMD(
    msg=True,
    timeLimit=60*4 
    ))

print("Solver Status:", pulp.LpStatus[prob.status])
print("Continuous Leave Days:", max_continuous_nonworking_days.value())

# PART II: Identifying How To Front-Loading Earnings

In this step, we consider our secondary goal of front-loading our earnings, ensuring that a greater portion of income is received earlier in the leave period. 

## Treat Amount of Continous Leave As A Constraint

In this step, we take our results from the first part of the problem and treat them as a constraint. 

In [ ]:
# Extract the results from the existing solution

max_continuous_nonworking_days_solution = max_continuous_nonworking_days.value()

# It is worth exploring whether relaxing the amount of continous leave taken
# would result in a solution that better allows us to front load our earnings.
# In order to do so, I am going to create a paramater for scaling down the amount of 
# continuous leave taken.

# This will be a parameter that I can adjust to see how it impacts the solution
continuous_leave_scaling_factor = 1.0 #should be between 0 and 1

# Add a constraint to ensure that the scaled continuous leave days are respected
prob += max_continuous_nonworking_days >= continuous_leave_scaling_factor * max_continuous_nonworking_days_solution, "Scaled_Continuous_Leave_Days"


## Redefine the Objective Function

In this section, I will define a new objective: maximizing the present value of salary earned. I will use some rough assumptions to calculate both how salary is earned and how its present value is calculated. Note that tax impacts, including any potential impacts of earning salary in one year versus another, are out of scope. 

In [ ]:
# Define cash flows associated with salary

# I am going to make rough assumptions about how the company calculates how much salary to pay each month
# While there may be some nuances to this, I do not believe they are important enough to warrant a more complex model

# Assumption 1: I will assume that salary can be calculated on a daily basis based on the number of working days
salary_annual = float(os.getenv("salary_annual")) #set in the configuration file
salary_daily = salary_annual / sum([is_company_workday[d] for d in dates]) #Approximation 

# Assumption 2: I will assume that salary is earned on all company workdays which are not used for unpaid leave
# This assumption is probably slightly inaccurate but good enough for my purposes
salary_fv = {d: pulp.LpVariable(f"salary_fv_{d.strftime('%Y%m%d')}", lowBound=0) for d in dates}

for d in dates:
    prob += salary_fv[d] == salary_daily * is_company_workday[d] * (1 - is_unpaid_fcl_leave[d]), f"salary_fv_{d.strftime('%Y%m%d')}"


In [ ]:
# Define present values of cash flows associated with salary

# Assumption 1: I will assume that present value can be calculated based on a daily linearized discount rate
# This is a simplification but will be good enough for my purposes

discount_rate_annual = 0.05 #5% discount rate
discount_rate_daily = (1 + discount_rate_annual)**(1/365) - 1

# I am going to assume that the discount rate is applied to the salary earned on each day
salary_pv = {d: pulp.LpVariable(f"salary_pv_{d.strftime('%Y%m%d')}", lowBound=0) for d in dates}
for d in dates:
    prob += salary_pv[d] == salary_fv[d] * (1 / (1 + discount_rate_daily)**((d - start_date).days)), f"salary_pv_{d.strftime('%Y%m%d')}"


In [ ]:
# Objective: Maximize the present value of salary cash flows
prob += pulp.lpSum([salary_pv[d] for d in dates]), "Maximize Present Value Salary"

## Solve & Export Results

In [ ]:
# Solve 
prob.solve(pulp.PULP_CBC_CMD(
    msg=True,
    timeLimit=60*4 
    ))

print("Solver Status:", pulp.LpStatus[prob.status])
print("Continuous Leave Days:", max_continuous_nonworking_days.value())
print("Salary Present Value:", round(sum([salary_pv[d].value() for d in dates]), 2))
print("Salary Future Value:", round(sum([salary_fv[d].value() for d in dates]), 2))

In [ ]:
# Create human-readable output: an excel document that shows the dates and the type of leave taken on each date

# Create a dataframe which has all dates
results = pd.DataFrame(dates, columns=["Date"])
results["day_of_week"] = [d.strftime('%A') for d in dates]
results["month"] = [d.strftime('%B') for d in dates]  # Will show "January", "February", etc.
results["week"] = results["Date"].dt.to_period("W").apply(lambda r: r.start_time)

# Add columns for each type of leave
results["is_company_holiday"] = [is_company_holiday[d] for d in dates]
results["is_weekend"] = [is_weekend[d] for d in dates]
results["is_std"] = [is_std[d] for d in dates]
results["is_paid_fcl"] = [is_paid_fcl[d].value() for d in dates]
results["is_unpaid_fcl_leave"] = [is_unpaid_fcl_leave[d].value() for d in dates]
results["is_vacation"] = [is_vacation[d].value() for d in dates]
results["is_optional_holiday"] = [is_optional_holiday[d].value() for d in dates]
results["is_non_absence"] = [is_non_absence[d].value() for d in dates]
results["is_not_working"] = [is_not_working[d].value() for d in dates]
results["is_company_workday"] = [is_company_workday[d] for d in dates]
results["is_day_accrual_eligible"] = [is_day_accrual_eligible[d].value() for d in dates]

# Quality control columns 
results["continuous_nonworking_days"] = [continuous_nonworking_days[d].value() for d in dates]
#results["salary_fv"] = [salary_fv[d].value() for d in dates]
#results["salary_pv"] = [salary_pv[d].value() for d in dates]
#results["z"] = [z[d].value() for d in dates]

# Sort the results by date
results = results.sort_values(by="Date")
# Reset the index
results.reset_index(drop=True, inplace=True)

# Save the data frame to an excel file
results.to_excel("parental_leave_schedule.xlsx", index=False)

In [ ]:
# Create human-readable weekly summary 
weekly_summary = results.groupby("week").agg({
    "is_std": "sum",
    "is_paid_fcl": "sum",
    "is_unpaid_fcl_leave": "sum",
    "is_vacation": "sum",
    "is_optional_holiday": "sum",
    "is_company_holiday": "sum",
    #"is_weekend": "sum",
    #"is_not_working": "sum",
    "is_non_absence": "sum"
}).reset_index()

# Rename columns for clarity
weekly_summary.rename(columns={
    "is_std": "Short-Term Disability Days",
    "is_paid_fcl": "Paid FCL Days",
    "is_unpaid_fcl_leave": "Unpaid FCL Days",
    "is_vacation": "Vacation Days",
    "is_optional_holiday": "Optional Holidays",
    "is_company_holiday": "Company Holidays",
    #"is_weekend": "Weekend Days",
    #"is_not_working": "Total Non-Working Days",
    "is_non_absence": "Total Non-Absence Days"
}, inplace=True)

In [ ]:
# Create human-readable monthly summary
monthly_summary = results.groupby(results["Date"].dt.to_period("M")).agg({
    "is_std": "sum",
    "is_paid_fcl": "sum",
    "is_unpaid_fcl_leave": "sum",
    "is_vacation": "sum",
    "is_optional_holiday": "sum",
    "is_company_holiday": "sum",
    "is_non_absence": "sum",
    "is_not_working": "sum",
    "is_company_workday": "sum",
    "is_day_accrual_eligible": "sum"
}).reset_index()

# Add a column for vacation accrual status
monthly_summary["is_vacation_accrued"] = monthly_summary["Date"].apply(
    lambda m: is_vacation_accrued_in_month[m].value() if m in is_vacation_accrued_in_month else None
)

monthly_summary["proportion_of_month_worked"] = monthly_summary["Date"].apply(
    lambda m: proportion_of_month_worked[m].value() if m in proportion_of_month_worked else None
)

# Rename columns for clarity
monthly_summary.rename(columns={
    "is_std": "Short-Term Disability Days",
    "is_paid_fcl": "Paid FCL Days",
    "is_unpaid_fcl_leave": "Unpaid FCL Days",
    "is_vacation": "Vacation Days",
    "is_optional_holiday": "Optional Holidays",
    "is_company_holiday": "Company Holidays",
    "is_non_absence": "Total Non-Absence Days",
    "is_not_working": "Total Non-Working Days",
    "is_company_workday": "Company Workdays",
    "is_vacation_accrued": "Vacation Accrued",
    "is_day_accrual_eligible": "Accrual Eligible Days"
}, inplace=True)

In [ ]:
# Save the detailed results and weekly summary to separate tabs in the same Excel file
with pd.ExcelWriter("parental_leave_schedule.xlsx", engine="openpyxl") as writer:
    # Save the detailed results to the first tab
    results.to_excel(writer, sheet_name="Detailed Results", index=False)
    
    # Save the weekly summary to the second tab
    weekly_summary.to_excel(writer, sheet_name="Weekly Schedule", index=False)

    # Save the monthly summary to the third tab
    monthly_summary.to_excel(writer, sheet_name="Monthly Schedule", index=False)